<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/ChatHaruhi_x_Qwen1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日 x Qwen 1.8B

本脚本是对千问1.8B模型的微调和测试，使得Qwen 1.8B能够有角色扮演的能力

This script fine-tunes and tests the Qwen 1.8B model to give Qwen 1.8B the capability of role playing.

- 118K训练数据由李鲁鲁收集，

- 模型是由[豆角](https://github.com/goodnessSZW)训练的

- Qwen inference代码由米唯实编写，

- 李鲁鲁编写了ChatHaruhi内部的prompt组织函数


### Chat凉宫春日 Chat-Haruhi-Suzumiya
### Reviving Anime Character in Reality via Large Language Model

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()
[![Huggingface Gradio](https://img.shields.io/static/v1?label=Demo&message=Huggingface%20Gradio&color=orange)](https://huggingface.co/spaces/silk-road/ChatHaruhi)

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，


<details>
  <summary> 本项目由李鲁鲁, 冷子昂, 闫晨曦, 封小洋, scixing, 沈骏一, Aria Fei, 王皓, 米唯实, 冷月, JunityZhan, 贾曜恺, 吴平宇, 孙浩甄等开发。 </summary>

本项目是一个开源项目，项目成员均在DataWhale等开源社区招募。

李鲁鲁( [Cheng Li@SenseTime](https://github.com/LC1332) )发起了整个项目,并设计和实现了项目的大多数功能。

冷子昂( [Ziang Leng@SenseTime](https://blairleng.github.io) )设计和实现了整体的ChatHaruhi1.0的训练,数据生成和后端架构。

闫晨曦( [Chenxi Yan@Chengdu University of Information Technology](https://github.com/todochenxi) )实现和维护了ChatHaruhi1.0版本的后端。

沈骏一( [Junyi Shen@Zhejiang University](https://github.com/J1shen) )实现了训练代码,参与了训练数据集生成。

王皓( [Hao Wang](https://github.com/wanghao07456) )收集了武林外传的台本数据,参与了增广数据的生成。

米唯实( [Weishi MI@Tsinghua University](https://github.com/hhhwmws0117) )参与了增广数据生成，进行了chatharuhi2.0的实现（with 李鲁鲁），并将其上传到了公用Pypi账号。

Yaying Fei( [Aria Fei@Beijing University of Technology](https://ariafyy.github.io/) )实现了台本工具 ASR 功能,参与了Openness-Aware Personality paper分支项目。

封小洋( [Xiaoyang Feng@Nanjing Agricultural University](https://github.com/fengyunzaidushi) )整合了台本识别工具功能,参与了Openness-Aware Personality paper分支项目。

冷月( [Song Yan](https://github.com/zealot52099) )收集了big bang thoery的数据。实现了台本格式转换功能。

scixing(汪好盛)( [HaoSheng Wang](https://github.com/ssccinng) )实现了台本工具中声纹识别功能,以及tts-vits语音合成功能。

Linkang Zhan( [JunityZhan@Case Western Reserve University](https://github.com/JunityZhan) ) 收集了原神的system prompt和故事数据。

贾曜恺( [Yaokai Jia](https://github.com/KaiJiaBrother) )实现了Vue版本的前端,并且在心理项目中实践了Bert的GPU抽取。

吴平宇( [Pingyu Wu@Juncai Shuyun](https://github.com/wpydcr) )帮助部署了第一版本的训练代码。

孙浩甄( [Haozhen Sun@Tianjin University] )绘制了ChatHaruhi角色的拼图。


</details>


Please star our project if you find helpful https://github.com/LC1332/Chat-Haruhi-Suzumiya

如果感觉有帮助的话欢迎star我们的github页面 https://github.com/LC1332/Chat-Haruhi-Suzumiya

# Qwen test

In [1]:
! pip install -q accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is no

不设置hf_key会报warning

现在colab需要在左侧设置HF_TOKEN 这个变量 （点小钥匙）

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("silk-road/Chat-Haruhi_qwen_1_8", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("silk-road/Chat-Haruhi_qwen_1_8", trust_remote_code=True).half().cuda()
model = model.eval()

tokenizer_config.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/Chat-Haruhi_qwen_1_8:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/2.04G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev


/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 1046 (delta 244), reused 306 (delta 210), pack-reused 693
Receiving objects: 100% (1046/1046), 106.35 MiB | 24.94 MiB/s, done.
Resolving deltas: 100% (574/574), done.
/content/Haruhi-2-Dev


In [5]:
!pip -q install openai tiktoken langchain datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency 

正常角色的测试

如果使用本地模型 建议llm = "foo"

In [6]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = 'haruhi', max_len_story = 1000, llm="foo" )

prompt = chatbot.generate_prompt(role='阿虚', text = '我看新一年的棒球比赛要开始了！我们要去参加吗？')

response, history = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

Generating train split: 0 examples [00:00, ? examples/s]

正在下载Luotuo-Bert


config.json:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

models.py:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

春日:「当然！我们一定要全力以赴，争取获得好成绩！」


默认llm是openai的 当然 generate_prompt 不会调用到openai

会抛出一句warning call LangChainGPT but openai key has not yet been set, use idle key instead

In [ ]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = 'haruhi', max_len_story = 1000 )

prompt = chatbot.generate_prompt(role='阿虚', text = '我看新一年的棒球比赛要开始了！我们要去参加吗？')

response, history = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

In [11]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = 'haruhi', max_len_story = 1000, llm="foo"  )

prompt = chatbot.generate_prompt(role='李鲁鲁', text = '春日你好啊，我是新同学李鲁鲁')

response, _ = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

春日:「李鲁鲁同学，你好。请问有什么我可以帮你的吗？」


In [13]:
prompt = chatbot.generate_prompt(role='李鲁鲁', text = '你知道我叫什么名字吗')

response, _ = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

春日:「当然，你叫李鲁鲁对吧」


奇怪角色的载入

目前支持 silk-road/ChatHaruhi-Waifu/女贤者

安莉娜， 艾希 和 艾希2

In [10]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_from_hf = 'silk-road/ChatHaruhi-Waifu/女贤者', max_len_story = 1000 , llm="foo" )

prompt = chatbot.generate_prompt(role='男子', text = '你已经不能动了')

response, _ = model.chat(tokenizer, prompt, history=[])
print(response)

chatbot.append_response(response)

Generating train split: 0 examples [00:00, ? examples/s]

女贤者:「你怎么能这样对我？我并没有做什么对不起你的事情。」


模型还没有进一步finetune，等finetune了之后继续发一个waifu版本的模型

In [ ]:
# import os
# import openai
# key = "key is not necessary here"
# key_bytes = key.encode()
# os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')